<a href="https://colab.research.google.com/github/BigData-MachineLearning/P-Set3/blob/main/Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Usamos la función pactan para llamar las bibliotecas necesarias para el análisis
install.packages('pacman')
library(pacman)
p_load(tidyverse,rio,tidymodels, glmnet)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’, ‘writexl’, ‘R.utils’



rio installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘Rcpp’, ‘diagram’, ‘lava’, ‘listenv’, ‘parallelly’, ‘prodlim’, ‘future’, ‘warp’, ‘iterators’, ‘lhs’, ‘DiceDesign’, ‘patchwork’, ‘globals’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’, ‘furrr’, ‘slider’, ‘foreach’, ‘GPfit’, ‘modelenv’, ‘dials’, ‘hardhat’, ‘infer’, ‘modeldata’, ‘parsnip’, ‘recipes’, ‘rsample’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’



tidymodels installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’, ‘config’, ‘tfautograph’, ‘reticulate’, ‘t

In [75]:
test <- import("/content/test_f.rds")
train <- import("/content/train_f.rds")

In [76]:
train["work_size"][is.na(train["work_size"])] <- 0
train["p5100"][is.na(train["p5100"])] <- 0

test["work_size"][is.na(test["work_size"])] <- 0
test["p5100"][is.na(test["p5100"])] <- 0


In [78]:
names(train)

[1] "id"              "clase"           "dominio"         "p5000"          
 [5] "p5010"           "p5090"           "p5100"           "p5130"          
 [9] "p5140"           "nper"            "npersug"         "ingtotug"       
[13] "ingtotugarr"     "ingpcug"         "li"              "lp"             
[17] "pobre"           "indigente"       "npobres"         "nindigentes"    
[21] "fex_c"           "depto"           "fex_dpto"        "mean_edad"      
[25] "jefe_mujer"      "educ_jefe"       "tasa_afil"       "reg_salud"      
[29] "num_adult"       "num_minors"      "num_ocup"        "cost_arriendo"  
[33] "cuartos_per"     "access_finan"    "bonificaciones"  "subs_alimeto"   
[37] "pay_alimento"    "pay_vivienda"    "pay_otros"       "primas"         
[41] "mean_hrs_work"   "work_size"       "tasa_cotizantes" "sub_empleo"     
[45] "disp_trabajar"   "dinero_trabajo"  "dinero_arriendo" "dinero_externo" 
[49] "dinero_remesas"  "ayuda_gob"

In [79]:
train <- train %>% select(p5000,p5010,p5090,p5100,p5100, nper, npersug,li,lp,mean_edad,jefe_mujer,educ_jefe,
                          tasa_afil,reg_salud,num_adult,num_minors,num_ocup,cost_arriendo,cuartos_per,
                          access_finan,bonificaciones,subs_alimeto,pay_alimento,pay_vivienda,pay_otros,primas,
                          mean_hrs_work,work_size,tasa_cotizantes,sub_empleo,disp_trabajar,dinero_trabajo,
                          dinero_arriendo, dinero_externo, dinero_remesas, ayuda_gob, ingtotugarr,lp)


In [80]:
train <- train |>
  mutate( ingtotugarr = as.numeric(ingtotugarr))


In [81]:
lasso_spec <- linear_reg(penalty = tune(), mixture = 1) |>
  set_mode("regression") |>
  set_engine("glmnet")

In [82]:
lasso_recipe <-
  recipe(formula = ingtotugarr ~ ., data = train) %>%
  step_novel(all_nominal_predictors()) %>%
  step_dummy(all_nominal_predictors()) %>%
  step_zv(all_predictors()) %>%
  step_normalize(all_predictors())

In [83]:
lasso_workflow <- workflow() %>%
  add_recipe(lasso_recipe) %>%
  add_model(lasso_spec)

In [84]:
penalty_grid <- grid_regular(penalty(range = c(-2, 2)), levels = 50)

In [85]:
lasso_folds <- vfold_cv(train, v = 10)

In [86]:
lasso_tune <- tune_grid(lasso_workflow,
  resamples = lasso_folds,
  grid = penalty_grid)

In [87]:
best_penalty <- select_best(lasso_tune, metric = "rmse")

In [88]:
best_penalty

penalty,.config
<dbl>,<chr>
0.01,Preprocessor1_Model01


In [89]:
lasso_final <- finalize_workflow(lasso_workflow, best_penalty)

lasso_final_fit <- fit(lasso_final, data = train)

In [90]:
summary(lasso_final_fit)

        Length Class      Mode   
pre     3      stage_pre  list   
fit     2      stage_fit  list   
post    1      stage_post list   
trained 1      -none-     logical

In [91]:
test<- augment(lasso_final_fit, new_data = test)


In [92]:
submission<- test |> select(id,lp,.pred) |>
  rename(ingtotugarr=.pred) |>
  mutate(pobre = ifelse(ingtotugarr<lp,1,0)) |>
  select(id,pobre)
submission

id,pobre
<chr>,<dbl>
2a7ddc2779480d7f19834953,0
a0c2e751e582fd49d564f308,0
57273d19e8464a5ff66a582b,0
418d052ff7878940ab938601,0
212a37fc17016a3c78f76852,0
7b0b8c4814944383d6c8cef1,0
5e02916a18573d43c00da7c7,0
927ccc0298577d7009ef94ea,0
428494ae91e022e32c269115,0


In [94]:
rio::export(submission, "/content/submission.csv")